In [1]:
# Import, preparation/processing and dataset - Diabetic Retinopathy 224x224 Gaussian Filtered
# Data loader, transformation
# NN, activation -> class, __init__
# Loss and optimizer
# Training loop
# Model evaluate

import os
import cv2
import numpy as np
from tqdm import tqdm

In [2]:
# Preprocessing
rebuild_data = False

class DR():
    No_DR = r"C:\Users\Gokul\Downloads\archive\gaussian_filtered_images\No_DR"
    Mild = r"C:\Users\Gokul\Downloads\archive\gaussian_filtered_images\Mild"
    Moderate = r"C:\Users\Gokul\Downloads\archive\gaussian_filtered_images\Moderate"
    Severe = r"C:\Users\Gokul\Downloads\archive\gaussian_filtered_images\Severe"
    Proliferate_DR = r"C:\Users\Gokul\Downloads\archive\gaussian_filtered_images\Proliferate_DR"
    classes = {No_DR:0, Mild:1, Moderate:2, Severe:3, Proliferate_DR:4}
    data = []
    count_0 = 0
    count_1 = 0
    count_2 = 0
    count_3 = 0
    count_4 = 0

    def make_data(self):
        for label in self.classes:
            for f in tqdm(os.listdir(label)):
                if "png" in f:
                    try:
                        path = os.path.join(label, f)   # join label and image
                        img = cv2.imread(path, 0)
                        self.data.append([np.array(img), [self.classes[label]]])

                        if label == self.No_DR:
                            self.count_0 += 1
                        elif label == self.Mild:
                            self.count_1 += 1
                        elif label == self.Moderate:
                            self.count_2 += 1
                        elif label == self.Severe:
                            self.count_3 += 1
                        elif label == self.Proliferate_DR:
                            self.count_4 += 1
                            
                    
                    except Exception as e:
                        pass

        np.random.shuffle(self.data)
        np.save("DR_data.npy", self.data)
        print("No_DR_count: ", DiabeticR.count_0)
        print("Mild_count: ", DiabeticR.count_1)
        print("Moderate_count: ", DiabeticR.count_2)
        print("Severe_count: ", DiabeticR.count_3)
        print("Proliferate_DR_count: ", DiabeticR.count_4)

In [3]:
if rebuild_data:
    DiabeticR = DR()
    DiabeticR.make_data()


data_processed = np.load("DR_data.npy", allow_pickle = True) 
print(len(data_processed))

3662


In [4]:
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as func
import torchvision.transforms as transforms

# Parameters
input_size = data_processed[0][0].shape[0]
batch_size = 128
no_classes = 5
no_epochs = 5
learning_rate = 0.001

In [5]:
# Data
images = torch.Tensor([i[0] for i in data_processed]).view(-1, input_size, input_size) #Images
images = images/255.0
labels = torch.Tensor([i[1] for i in data_processed])                                  #Labels


In [6]:
# Data split
split_percent = 0.1
split_size = int(len(data_processed)*split_percent)
train_x = images[:-split_size]
train_y = labels[:-split_size]
test_x = images[-split_size:]
test_y = labels[-split_size:]

In [7]:
# NN
class DRmodel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,5)      # 1x224x224  -> 32x220x220(conv1) -> 32x110x110(pool)
        self.norm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32,64,5)     # 32x110x110 -> 64x106x106(conv2) -> 64x53x53  (pool)
        self.norm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64,128,5)    # 64x53x53   -> 128x49x49 (conv3) -> 128x24x24 (pool)
        self.norm3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128,256,5)   # 128x24x24  -> 256x20x20 (conv4) -> 256x10x10 (pool)
        self.norm4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256,512,5)   # 256x10x10  -> 512x6x6   (conv5) -> 512x3x3   (pool)
        self.norm5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(2,2)
        
        self.fc1 = nn.Linear(512*3*3, 1024)
        self.fc2 = nn.Linear(1024, 400)
        self.fc3 = nn.Linear(400, 120)
        self.fc4 = nn.Linear(120, 84) 
        self.fc5 = nn.Linear(84,no_classes)

    def forward(self,x):
        x = self.pool(func.relu(self.conv1(x)))
        x = self.norm1(x)
        x = self.pool(func.relu(self.conv2(x)))
        x = self.norm2(x)
        x = self.pool(func.relu(self.conv3(x)))
        x = self.norm3(x)
        x = self.pool(func.relu(self.conv4(x)))
        x = self.norm4(x)
        x = self.pool(func.relu(self.conv5(x))) 
        x = self.norm5(x)
        x = x.view(-1, 512*3*3)
        x = func.relu(self.fc1(x))
        x = func.relu(self.fc2(x))
        x = func.relu(self.fc3(x))
        x = func.relu(self.fc4(x)) 
        x = self.fc5(x)
        return x

model = DRmodel()
print(model)

# Loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

DRmodel(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (norm3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1))
  (norm4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1))
  (norm5): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4608, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=400, bias=True)
  (fc3): Lin

In [8]:
# Training loop
for epoch in range(no_epochs):
    for data in tqdm(range(0, len(train_x), batch_size)):
        train_img = train_x[data: data + batch_size].view(-1, 1, input_size, input_size)
        train_label = train_y[data: data + batch_size]

        # forward and loss
        output = model(train_img)
        loss = loss_func(output, train_label.squeeze().type(torch.LongTensor))

        # backprop and updation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 

100%|██████████| 26/26 [06:04<00:00, 14.00s/it]


In [9]:
# Evaluate
with torch.no_grad():
    correct = 0
    total = 0
    for i in tqdm(range(len(test_x))):
        real = torch.argmax(test_y[i])
        output = model(test_x[i].view(-1, 1, input_size, input_size))
        predicted = torch.argmax(output)

        if predicted == real:
            correct += 1
        total += 1

print("Accuracy: ", round(correct/total, 3))

100%|██████████| 366/366 [00:31<00:00, 11.68it/s]

Accuracy:  0.883
